In [63]:
import numpy as np
import pandas as pd
import os
import re

[Outil pour le Regex :](https://pythex.org/)

In [75]:
regex_date = "(../../....|..-..-....)"
regex_date_litteral = "([0-9]{1,2} *(janvier|f(e|é)vrier|mars|avril|mai|juin|juillet|ao(u|û)t|septembre|octobre|novembre|d(e|é)cembre) *[0-9]{2,4})"

In [20]:
train_folder = "train_folder_predilex/train_folder/"
test_folder = "test_folder_predilex/test_folder/"
train_path = "train_folder_predilex/train_folder/txt_files/"
test_path = "test_folder_predilex/test_folder/txt_files/"

In [22]:
df_train_y = pd.read_csv("y_train_predilex.csv")
df_train_x = pd.read_csv("train_folder_predilex/train_folder/x_train_ids.csv")
df_train = df_train_x.set_index('ID').join(df_train_y.set_index('ID'))
df_train.head()

,filename,sexe,date_accident,date_consolidation
ID,,,,
0,Agen_100515.txt,homme,1991-04-09,n.c.
1,Agen_1100752.txt,homme,2005-06-10,2010-01-19
2,Agen_1613.txt,femme,1997-09-26,n.c.
3,Agen_2118.txt,femme,1982-08-07,1982-11-07
4,Agen_21229.txt,homme,1996-11-26,n.c.


In [23]:
df_test_y = pd.read_csv("y_test_pred.csv")
df_test_x = pd.read_csv("test_folder_predilex/test_folder/x_test_ids.csv")
df_test = df_test_x.set_index('ID').join(df_test_y.set_index('filename'))

In [24]:
def read_properly(path,filename):
    f = open(os.path.join(path,filename),"r",encoding='utf-8')
    return f.read().replace('\xa0','').replace('\n',' ').replace('’',' ')\
                 .replace(',',' ').replace('...','___').replace('.',' ')\
                 .replace(':',' ').replace('(',' ').replace(')',' ')

In [25]:
df_train['contenu'] = [read_properly(train_path,i) for i in df_train.filename]

In [26]:
df_test['contenu'] = [read_properly(test_path,i) for i in df_test.filename]

In [27]:
def frequency_word(text,word):
    return sum(1 for i in text.split() if i.lower()==word)

In [74]:
def frequency_regex(text,regex):
    return re.findall(regex,text)

['19/02/1997', '05-07-1996']

In [28]:
df_train['CC_elle']=[frequency_word(text,"elle") for text in df_train.contenu]
df_train['CC_il']=[frequency_word(text,"il") for text in df_train.contenu]
df_train['CC_madame']=[frequency_word(text,"madame") for text in df_train.contenu]
df_train['CC_monsieur']=[frequency_word(text,"monsieur") for text in df_train.contenu]
df_train['CC_mme']=[frequency_word(text,"mme") for text in df_train.contenu]
df_train['CC_m']=[frequency_word(text,"m") for text in df_train.contenu]

In [29]:
df_test['CC_elle']=[frequency_word(text,"elle") for text in df_test.contenu]
df_test['CC_il']=[frequency_word(text,"il") for text in df_test.contenu]
df_test['CC_madame']=[frequency_word(text,"madame") for text in df_test.contenu]
df_test['CC_monsieur']=[frequency_word(text,"monsieur") for text in df_test.contenu]
df_test['CC_mme']=[frequency_word(text,"mme") for text in df_test.contenu]
df_test['CC_m']=[frequency_word(text,"m") for text in df_test.contenu]

In [57]:
df_train['Homme'] = df_train.CC_monsieur+df_train.CC_m+df_train.CC_il
df_train['Femme'] = df_train.CC_madame+df_train.CC_mme+df_train.CC_elle

In [58]:
df_test['Homme'] = df_test.CC_monsieur+df_test.CC_m+df_test.CC_il
df_test['Femme'] = df_test.CC_madame+df_test.CC_mme+df_test.CC_elle

In [59]:
gender_list = []
Homme = df_train.Homme.tolist()
Femme = df_train.Femme.tolist()
for i in range(len(Homme)):
    if Homme[i]>Femme[i]:
        gender_list.append('homme')
    elif Femme[i]>Homme[i]:
        gender_list.append('femme')
    else:
        gender_list.append('neutre')
        print('NEUTRE TROUVE')

df_train['Genre'] = gender_list

NEUTRE TROUVE
NEUTRE TROUVE
NEUTRE TROUVE


In [60]:
gender_list = []
Homme = df_test.Homme.tolist()
Femme = df_test.Femme.tolist()
for i in range(len(Homme)):
    if Homme[i]>Femme[i]:
        gender_list.append('homme')
    elif Femme[i]>Homme[i]:
        gender_list.append('femme')
    else:
        gender_list.append('neutre')
        print('NEUTRE TROUVE')

df_test['Genre'] = gender_list

NEUTRE TROUVE


In [61]:
accuracy = np.sum(df_train.sexe==df_train.Genre)/df_train.shape[0]
accuracy*100

89.87012987012987

In [62]:
accuracy = np.sum(df_test.sexe==df_test.Genre)/df_test.shape[0]
accuracy*100

64.59143968871595

In [77]:
df_train.sexe.value_counts()

homme    559
femme    206
n.c.       5
Name: sexe, dtype: int64

In [78]:
df_test.sexe.value_counts()

homme    184
femme     73
Name: sexe, dtype: int64